# マークルルート
ブロックヘッダにはブロックに格納されているトランザクションを要約した値であるマークルルートというフィールドが32バイト存在する。
## マークルルートの復習
`マークルルート`は、あるブロックないに格納されているトランザクションデータを要約した32バイトのデータのこと。  
トランザクションデータを並べ、隣り合ったデータをセットにしつつ、Double-SHA256でハッシュ化し、ハッシュ値が１つだけになるまで繰り返す。  
この時にハッシュ値によって形成されるデータ構造を`マークルツリー`と呼ぶ。  
マークルルートを計算する際に、トランザクションデータ数の合計が奇数の場合は最後のデータを複製して合計を偶数にすることで計算を行う.  
また、特定のトランザクションデータを検証する際に用いられるハッシュ値の組み合わせを`マークルパス`と呼ぶ。
## マークルルートとマークルツリーの意義
マークルルートとマークルツリーは、トランザクションデータがブロックに格納されているかどうかの検証を容易にするために実装された。  
ビットコインをはじめとするブロックチェーンでは、フルノードを立てれば単体でもトランザクションやブロックの正しさを検証することが可能だが、SPVノードの場合、フルノードに問い合わせることでトランザクションの正しさを検証する。  
この時に利用されるのが、マークルルートで、実際の検証の際はマークルルートとマークルパスを利用する。  
検証したい時は当該トランザクションデータのDouble-SHA256を計算し、その後、マークルルートとマークルパスを問い合わせて入手する。  
TX3を検証したい時に必要なマークルパスは「H4, H12, H5677」となり、算出したトランザクションのハッシュ値とマークルパスを計算していきハッシュ値を算出する。  
ここで算出されたハッシュ値とマークルルートが一致するかを検証し、同じであれば当該トランザクションは確かにブロックに格納されている正しいデータであると証明できる。  
## マークルルートの実装方針
マークルルートを実装する際の基本方針は以下の通り  
- トランザクションデータを整列させる
- トランザクションデータ数の合計が奇数の倍は最後のデータを複製する
- 先頭から順番に２つずつまとめてハッシュ化する
- 最後の１つになるまで、まとめてハッシュ化を繰り返す  
マークルルートの算出において'肝'になるポイントは、トランザクションデータの規定しておくこと。  
ハッシュ関数は入力値が少しでも変わると出力されるデータが大きく変わるので、格納されているトランザクションデータの内容が同じでも、順番が異なるだけで結果が大きく変わってしまう。  
## マークルルートを計算してみる
以下の11個のトランザクションデータからマークルルートを算出する

In [ ]:
[
"e7c6a5c20318e99e7a2fe7e9c534fae52d402ef6544afd85a0a1a22a8d09783a",
"3fe7ac92b9d20c9b5fb1ba21008b41eb1208af50a7021694f7f73fd37e914b67",
"b3a37d774cd5f15be1ee472e8c877bcc54ab8ea00f25d34ef11e76a17ecbb67c",
"dcc75a59bcee8a4617b8f0fc66d1444fea3574addf9ed1e0631ae85ff6c65939",
"59639ffc15ef30860d11da02733c2f910c43e600640996ee17f0b12fd0cb51e9",
"0e942bb178dbf7ae40d36d238d559427429641689a379fc43929f15275a75fa6",
"5ea33197f7b956644d75261e3c03eefeeea43823b3de771e92371f3d630d4c56",
"55696d0a3686df2eb51aae49ca0a0ae42043ea5591aa0b6d755020bdb64887f6",
"2255724fd367389c2aabfff9d5eb51d08eda0d7fed01f3f9d0527693572c08f6",
"c8329c18492c5f6ee61eb56dab52576b1de48bbb1d7f6aa7f0387f9b3b63722e",
"34b7f053f77406456676fdd3d1e4ac858b69b54daf3949806c2c92ca70d3b88d"
]

In [1]:
import hashlib

def sha256(data):
    return hashlib.sha256(data.encode()).hexdigest()

class MerkleTree():
    # まず tx_list を定義
    def __init__(self, tx_list):
        self.tx_list = tx_list

    # マークルルートを算出する処理
    def calc_merkleroot(self):
        # トランザクションデータのリストをtxsに格納。
        txs = self.tx_list
        # もし、トランザクションデータが１つだけの時、
        if len(txs) == 1:
            # その１つを返す。
            return txs[0]
        # 配列のデータが１つだけになるように計算していく
        while len(txs) > 1:
            # txtの数が奇数の場合、
            if len(txs) % 2 == 1:
                # 最後の要素をコピーして追加
                txs.append(txs[-1])
            # ハッシュ値を格納する空の配列を用意
            hashes = []
            # 隣り合う要素を結合してハッシュ化する処理を行う。
            # for文によって、0から配列の要素数まで2つ刻み、つまり要素数の半分の回数で繰り返し処理を行う。
            # これによって先頭から2つずつセットにしてハッシュ値を計算する
            for i in range(0, len(txs), 2):
                # 引数に与えられた配列の隣り合う要素についてjoinメソッドを離礁して結合したのち、sha256でハッシュ化した値を先ほど用意したhashesの配列に追加
                hashes.append(sha256("".join(txs[i:i + 2])))
            # txsをhashesにし、繰り返す
            txs = hashes
        return txs[0]

if __name__ == "__main__":
    txs = [
        "e7c6a5c20318e99e7a2fe7e9c534fae52d402ef6544afd85a0a1a22a8d09783a",
        "3fe7ac92b9d20c9b5fb1ba21008b41eb1208af50a7021694f7f73fd37e914b67",
        "b3a37d774cd5f15be1ee472e8c877bcc54ab8ea00f25d34ef11e76a17ecbb67c",
        "dcc75a59bcee8a4617b8f0fc66d1444fea3574addf9ed1e0631ae85ff6c65939",
        "59639ffc15ef30860d11da02733c2f910c43e600640996ee17f0b12fd0cb51e9",
        "0e942bb178dbf7ae40d36d238d559427429641689a379fc43929f15275a75fa6",
        "5ea33197f7b956644d75261e3c03eefeeea43823b3de771e92371f3d630d4c56",
        "55696d0a3686df2eb51aae49ca0a0ae42043ea5591aa0b6d755020bdb64887f6",
        "2255724fd367389c2aabfff9d5eb51d08eda0d7fed01f3f9d0527693572c08f6",
        "c8329c18492c5f6ee61eb56dab52576b1de48bbb1d7f6aa7f0387f9b3b63722e",
        "34b7f053f77406456676fdd3d1e4ac858b69b54daf3949806c2c92ca70d3b88d"
    ]

    mt = MerkleTree(txs)

    print(mt.calc_merkleroot())

45ce9219fbff637dbe398f21c765081c511d54b9758755875e764f488c21cfc2
